<a href="https://colab.research.google.com/github/eunjaelim/notefindProject/blob/master/EMNIST_PRJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mnist - 숫자

# 데이터 로드 / 전처리

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam







In [2]:
# CSV 파일 로드
train_df = pd.read_csv("emnist-digits-train.csv", header=None)
test_df = pd.read_csv("emnist-digits-test.csv", header=None)


In [3]:
test_df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39996,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39997,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39998,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
print(train_df.shape)

(240000, 785)


In [5]:
print(test_df.shape)

(40000, 785)


In [6]:
print(train_df.head())

   0    1    2    3    4    5    6    7    8    9    ...  775  776  777  778  \
0    8    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1    9    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
2    6    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    3    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    6    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   779  780  781  782  783  784  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 785 columns]


In [7]:
print(test_df.head())

   0    1    2    3    4    5    6    7    8    9    ...  775  776  777  778  \
0    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1    9    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
2    7    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    9    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    2    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   779  780  781  782  783  784  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 785 columns]


In [8]:
# 첫 번째 열은 라벨, 나머지 열은 이미지 픽셀 데이터
X_train = train_df.iloc[:, 1:].values
y_train = train_df.iloc[:, 0].values
X_test = test_df.iloc[:, 1:].values
y_test = test_df.iloc[:, 0].values

In [9]:
# 결측값 확인
print(train_df.isnull().sum())
print(test_df.isnull().sum())

0      0
1      0
2      0
3      0
4      0
      ..
780    0
781    0
782    0
783    0
784    0
Length: 785, dtype: int64
0      0
1      0
2      0
3      0
4      0
      ..
780    0
781    0
782    0
783    0
784    0
Length: 785, dtype: int64


In [10]:
# 훈련 데이터의 첫 번째 열은 라벨, 나머지는 이미지 데이터
X_train = train_df.iloc[:, 1:].values  # 이미지 데이터
y_train = train_df.iloc[:, 0].values   # 라벨

# 테스트 데이터도 동일하게 분리
X_test = test_df.iloc[:, 1:].values
y_test = test_df.iloc[:, 0].values

In [11]:
# 0~255 사이의 픽셀 값을 0~1 사이로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [12]:
# CNN을 위한 이미지 형태로 변경 (28x28 크기의 이미지로 재구성, 흑백 채널 추가)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [13]:
# y_train, y_test가 원래 라벨인지 확인 후, 필요 시 원핫 인코딩
if len(y_train.shape) > 2:  # 3차원 라벨일 경우 차원 축소
    y_train = y_train.argmax(axis=-1)
    y_test = y_test.argmax(axis=-1)

In [14]:
print("y_train final shape:", y_train.shape)  # (샘플 수, 10)
print("y_test final shape:", y_test.shape)

y_train final shape: (240000,)
y_test final shape: (40000,)


In [15]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [16]:
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

y_train shape: (240000, 10)
y_test shape: (40000, 10)


# CNN




### CNN 모델 설계

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# 모델 구조
model = Sequential()

# 첫 번째 Conv2D 층
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 두 번째 Conv2D 층
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 세 번째 Conv2D 층
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten 및 Dense 층 추가
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# 출력층 (10개의 클래스)
model.add(Dense(10, activation='softmax'))  # digit 데이터는 10개의 클래스

# 학습률을 0.001로 설정한 Adam 옵티마이저
optimizer = Adam(learning_rate=0.001)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 요약 출력
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 3, 3, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 469,642 (1.79 MB)

 Trainable params: 469,642 (1.79 MB)

 Non-trainable params: 0 (0.00 B)

### 모델 컴파일

# 모델 학습

In [ ]:
# 모델 학습
# 모델 학습
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=50,  # 학습할 epoch 수 (조정 가능)
                    batch_size=128)  # 한 번에 처리할 배


Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9222 - loss: 0.2440 - val_accuracy: 0.9893 - val_loss: 0.0375
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9896 - loss: 0.0357 - val_accuracy: 0.9906 - val_loss: 0.0321
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9917 - loss: 0.0276 - val_accuracy: 0.9919 - val_loss: 0.0281
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9934 - loss: 0.0222 - val_accuracy: 0.9931 - val_loss: 0.0258
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9947 - loss: 0.0184 - val_accuracy: 0.9931 - val_loss: 0.0256
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9955 - loss: 0.0146 - val_accuracy: 0.9932 - val_loss: 0.0270
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9964 - loss: 0.0117 - val_accuracy: 0.9935 - val_loss: 0.0271
Epoch 8/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9969 - loss: 0

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

### 모델 평가

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {test_acc}")

### 모델 예측

In [ ]:
# 새로운 데이터로 예측
predictions = model.predict(X_test[:5])

# 첫 번째 테스트 데이터 예측 결과 확인
print("예측 결과:", predictions.argmax(axis=1))
print("실제 라벨:", y_test[:5].argmax(axis=1))

In [ ]:
import matplotlib.pyplot as plt

# 예측 결과 시각화 함수
def visualize_predictions(images, true_labels, predicted_labels, num_samples=5):
    plt.figure(figsize=(10, 5))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(images[i].reshape(28, 28), cmap='gray')  # 28x28 크기의 흑백 이미지
        plt.title(f"True: {true_labels[i].argmax()}, Pred: {predicted_labels[i].argmax()}")
        plt.axis('off')  # 축 제거
    plt.show()

# 예측한 라벨 중 상위 5개 이미지와 라벨을 시각화
visualize_predictions(X_test, y_test, predictions, num_samples=5)

In [ ]:
model.save('my_digit_model.keras')

print("모델이 'my_digit_model.keras' 파일로 저장되었습니다.")